In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
import scipy.io
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'GeForce GTX 1050'

In [2]:
data = scipy.io.loadmat('DataSet_Total_Walsh.mat')

In [3]:
df = pd.DataFrame(data['dados'])
df.head()
# features: 0->29 30-features
# labels: 30 -> 35 6-labels

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-0.177237,0.099807,0.115007,1.564971,-3.426974,-0.701468,0.664442,-1.415147,-1.048870,-0.387557,...,0.796155,-0.035896,-0.476052,-0.233128,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.227044,0.079490,0.211965,1.189603,-3.436533,0.012993,0.433340,-1.663144,-1.222274,0.090048,...,1.202891,-0.217979,-0.221793,0.066452,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.204134,-0.081315,0.241718,4.302551,-4.435533,-1.315352,0.124246,-2.784308,-1.944363,-0.534279,...,2.709597,0.445616,-0.775808,0.274888,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.218254,-0.043448,0.238516,3.058132,-4.066913,-0.318107,0.404549,-2.426833,-1.486463,-0.308362,...,1.708945,0.200189,-0.347463,-0.009817,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.153955,-0.189749,0.240992,8.871455,-2.073549,-1.002858,0.253458,-4.489565,-0.735555,-0.508349,...,3.147622,0.846095,-0.810594,0.994890,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
## Perform Train/Test/Split
X = df.drop([30,31,32,33,34,35],axis=1).values
y = df[[30,31,32,33,34,35]].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=33, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.20,random_state=33, shuffle = True)

scaler = MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
X_val = torch.FloatTensor(X_val).cuda()
#y_train = torch.FloatTensor(y_train).cuda()
#y_test = torch.FloatTensor(y_test).cuda()

y_val = torch.LongTensor(y_val).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [6]:
class Model(nn.Module):
    def __init__(self, in_features=30, h1=100, h2=50, out_features=6):
        super().__init__()
        
        # input layer -> 1 hidden -> 2 hidden -> output 
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        # Pass the neuron input through its activation function to obtain the output of the neuron
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [7]:
torch.manual_seed(4)
model = Model().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
epochs = 1000 # number of runs through the training data
losses = []

for i in range(epochs):
    i+=1
    y_pred = model(X_train)
    loss = criterion(y_pred,torch.max(y_train, 1)[1] )
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%100 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad() #zero-grad to not accomulate the gradient over the epochs
    loss.backward()
    optimizer.step()

epoch:  1  loss: 1.79269266
epoch: 101  loss: 0.70871848
epoch: 201  loss: 0.32661030
epoch: 301  loss: 0.20047401
epoch: 401  loss: 0.14123903
epoch: 501  loss: 0.10857904
epoch: 601  loss: 0.08948430
epoch: 701  loss: 0.07470568
epoch: 801  loss: 0.06552012
epoch: 901  loss: 0.05650461


In [9]:
Test_Predictions = torch.max(y_pred,1)[1]
print(Test_Predictions)

tensor([4, 1, 2,  ..., 5, 1, 3], device='cuda:0')


# Validation Set

In [10]:
with torch.no_grad():
    y_val_pred = model.forward(X_val)

In [11]:
y_val_pred = torch.max(y_val_pred,1)[1]

In [12]:
y_val = y_val.cpu().numpy()
y_val_pred = y_val_pred.cpu().numpy()
y_val = np.argmax(y_val, axis=1) # assuming you have n-by-5 class_prob
print(confusion_matrix(y_val, y_val_pred))

[[1862    8    5    4    4    2]
 [   9 1806    2    0    6    2]
 [   0    0 1809    6    0    1]
 [   2    1    9 1849    9    7]
 [   3    6   17   12 1866    3]
 [  10    8    5   14    0 1850]]


In [13]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1885
           1       0.99      0.99      0.99      1825
           2       0.98      1.00      0.99      1816
           3       0.98      0.99      0.98      1877
           4       0.99      0.98      0.98      1907
           5       0.99      0.98      0.99      1887

   micro avg       0.99      0.99      0.99     11197
   macro avg       0.99      0.99      0.99     11197
weighted avg       0.99      0.99      0.99     11197



# Test Set

In [14]:
with torch.no_grad():
    y_test_pred = model.forward(X_test)

In [15]:
y_test_pred = torch.max(y_test_pred,1)[1]

In [16]:
y_test = y_test.cpu().numpy()
y_test_pred = y_test_pred.cpu().numpy()
y_test = np.argmax(y_test, axis=1) # assuming you have n-by-5 class_prob
print(confusion_matrix(y_test, y_test_pred))

[[2296    9    8    3    4    2]
 [  17 2318    9    3   15    6]
 [   0    0 2386    5    0    0]
 [   2    0    8 2348    7    6]
 [   9   11   15    7 2167    6]
 [   6    5    5   19    0 2294]]


In [17]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2322
           1       0.99      0.98      0.98      2368
           2       0.98      1.00      0.99      2391
           3       0.98      0.99      0.99      2371
           4       0.99      0.98      0.98      2215
           5       0.99      0.98      0.99      2329

   micro avg       0.99      0.99      0.99     13996
   macro avg       0.99      0.99      0.99     13996
weighted avg       0.99      0.99      0.99     13996

